In [4]:
def CustomParser(data):
    import json
    j1 = json.loads(data)
    return j1

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
df = pd.read_json('boardgames_100.json')

df.head()
id_to_idx = {} # A useful dictionary for converting between IDs and indices.  
for i, id in enumerate(df.loc[:,'id']):
    id_to_idx[id] = i

In [12]:
df['recommendations'] = df['recommendations'].apply(lambda x: x['fans_liked'])

In [14]:
adj_matrix = np.zeros((100,100))

In [22]:
for index in range(100):
    #print(f"{index} : {df.iloc[index]['recommendations']}")
    # Construct the graph
    v = id_to_idx[df.iloc[index]['id']] # Source node
    for endpoint in df.iloc[index]['recommendations']:
        try:
            w = id_to_idx[endpoint] # Sink node
            # Construct a directed edge (v, w)
            adj_matrix[v][w] = 1
        except:
            print(f"ID: {endpoint} not in top 100, discarded")

ID: 302723 not in top 100, discarded
ID: 245655 not in top 100, discarded
ID: 216734 not in top 100, discarded
ID: 233247 not in top 100, discarded
ID: 82222 not in top 100, discarded
ID: 169426 not in top 100, discarded
ID: 150376 not in top 100, discarded
ID: 256916 not in top 100, discarded
ID: 175640 not in top 100, discarded
ID: 286749 not in top 100, discarded
ID: 142379 not in top 100, discarded
ID: 233398 not in top 100, discarded
ID: 265188 not in top 100, discarded
ID: 163068 not in top 100, discarded
ID: 197376 not in top 100, discarded
ID: 302723 not in top 100, discarded
ID: 198928 not in top 100, discarded
ID: 245655 not in top 100, discarded
ID: 224037 not in top 100, discarded
ID: 181304 not in top 100, discarded
ID: 146508 not in top 100, discarded
ID: 2511 not in top 100, discarded
ID: 239188 not in top 100, discarded
ID: 178900 not in top 100, discarded
ID: 30549 not in top 100, discarded
ID: 218074 not in top 100, discarded
ID: 156129 not in top 100, discarded
ID: 1

In [23]:
adj_matrix

array([[0., 0., 1., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.]])